In [1]:
import os
import findspark

!chmod u=rwx,g=rx,o=rx  start.sh && ./start.sh -p "zTAgUdbQ&"
!chmod u=rwx,g=rx,o=rx  ngrok_install.sh && ./ngrok_install.sh -p "zTAgUdbQ&"

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "./spark-3.1.2-bin-hadoop2.7"

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

#get_ipython().system_raw('./ngrok config add-authtoken YOUR_AUTHTOKEN')
get_ipython().system_raw('./ngrok http 4050 &')

[sudo] password for thiago: Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://download.docker.com/linux/ubuntu focal InRelease
Hit:3 https://apt.releases.hashicorp.com focal InRelease
Hit:4 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Err:6 https://ngrok-agent.s3.amazonaws.com buster InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 0E61D3BBAAEE37FE
Reading package lists...
W: GPG error: https://ngrok-agent.s3.amazonaws.com buster InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 0E61D3BBAAEE37FE
E: The repository 'https://ngrok-agent.s3.amazonaws.com buster InRelease' is no

23/07/17 21:04:25 WARN Utils: Your hostname, W10-tks resolves to a loopback address: 127.0.1.1; using 172.25.91.212 instead (on interface eth0)
23/07/17 21:04:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/17 21:04:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","ID":"2f3751eec5bf38a1542392e7e439e96a","uri":"/api/tunnels/command_line","public_url":"https://681a-201-37-95-158.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [22]:

import zipfile

get_ipython().system_raw('wget https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/empresas.zip -q')
zipfile.ZipFile('./empresas.zip', 'r').extractall('./data')

get_ipython().system_raw('wget https://caelum-online-public.s3.amazonaws.com/2273-introducao-spark/01/socios.zip -q')
zipfile.ZipFile('./socios.zip', 'r').extractall('./data')


In [23]:
path = './data/empresas/part-0000*csv'
empresas = spark.read.csv(path, sep=';', inferSchema=True)

path = './data/socios/part-0000*csv'
socios = spark.read.csv(path, sep=';', inferSchema=True)


In [24]:
empresasColNames = [
    'cnpj_basico', 
    'razao_social_nome_empresarial', 
    'natureza_juridica', 
    'qualificacao_do_responsavel', 
    'capital_social_da_empresa', 
    'porte_da_empresa', 
    'ente_federativo_responsavel'
]

for index, colName in enumerate(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}", colName)
    
# convertendo string pra double
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f
empresas = empresas.withColumn('capital_social_da_empresa', f.regexp_replace('capital_social_da_empresa', ',', '.'))
empresas = empresas.withColumn('capital_social_da_empresa', empresas['capital_social_da_empresa'].cast(DoubleType()))

empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [25]:
sociosColNames = [
    'cnpj_basico', 
    'identificador_de_socio', 
    'nome_do_socio_ou_razao_social', 
    'cnpj_ou_cpf_do_socio', 
    'qualificacao_do_socio', 
    'data_de_entrada_sociedade', 
    'pais', 
    'representante_legal', 
    'nome_do_representante', 
    'qualificacao_do_representante_legal', 
    'faixa_etaria'
]

for index, colName in enumerate(sociosColNames):
    socios = socios.withColumnRenamed(f"_c{index}", colName)
    

socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [8]:
empresas.select('*')

DataFrame[cnpj_basico: int, razao_social_nome_empresarial: string, natureza_juridica: int, qualificacao_do_responsavel: int, capital_social_da_empresa: string, porte_da_empresa: int, ente_federativo_responsavel: string]

In [9]:
empresas.select('*').show(5, False)

+-----------+--------------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial                                                               |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+--------------------------------------------------------------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|306        |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                         |0,00                     |1               |null                       |
|1355       |BRASILEIRO & OLIVEIRA LTDA                                                                 

In [10]:
empresas.select('natureza_juridica', 'porte_da_empresa', 'capital_social_da_empresa').show(5, False)

+-----------------+----------------+-------------------------+
|natureza_juridica|porte_da_empresa|capital_social_da_empresa|
+-----------------+----------------+-------------------------+
|2240             |1               |0,00                     |
|2062             |5               |0,00                     |
|3034             |5               |0,00                     |
|2135             |5               |0,00                     |
|2062             |1               |4000,00                  |
+-----------------+----------------+-------------------------+
only showing top 5 rows



In [26]:
socios.select('*').show(5,False)

+-----------+----------------------+-------------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social  |cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-------------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|411        |2                     |LILIANA PATRICIA GUASTAVINO    |***678188**         |22                   |19940725                 |null|***000000**        |null                 |0                                  |7           |
|411        |2                     |CRISTINA HUNDERTMARK        

In [29]:
socios.select('nome_do_socio_ou_razao_social', 'faixa_etaria').show(5, False)

+-------------------------------+------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|
+-------------------------------+------------+
|LILIANA PATRICIA GUASTAVINO    |7           |
|CRISTINA HUNDERTMARK           |7           |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |
|EDUARDO BERRINGER STEPHAN      |5           |
|HANNE MAHFOUD FADEL            |8           |
+-------------------------------+------------+
only showing top 5 rows



In [34]:
# convertendo string pra double
from pyspark.sql.types import StringType
from pyspark.sql import functions as f
socios = socios.withColumn('data_de_entrada_sociedade', f.to_date(socios['data_de_entrada_sociedade'].cast(StringType()), 'yyyyMMdd'))
socios.select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada')).show(5, False)

+-------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_de_entrada|
+-------------------------------+------------+--------------+
|LILIANA PATRICIA GUASTAVINO    |7           |1994          |
|CRISTINA HUNDERTMARK           |7           |1994          |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |1994          |
|EDUARDO BERRINGER STEPHAN      |5           |1994          |
|HANNE MAHFOUD FADEL            |8           |1994          |
+-------------------------------+------------+--------------+
only showing top 5 rows

